# Import important packages

In [16]:
# %load rag.py
from IPython.display import Markdown, display

from operator import itemgetter

from langchain_community.llms.ollama import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.output_parsers import StrOutputParser

from langchain.text_splitter import CharacterTextSplitter

from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate

In [3]:
# Model
MODEL = "llama2"

In [4]:
chain = Ollama(model=MODEL) | StrOutputParser()
embeddings = OllamaEmbeddings()

## Load PDF File

In [5]:
# Load PDF file
filename = "Docker Deep Drive.pdf"
loader = PyPDFLoader(filename)
pages = loader.load_and_split()

In [17]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
documents = text_splitter.split_documents(loader.load())

In [6]:
template = """
    Answer the question based on the context below.
    If you can't answer the question, reply "I don't know."

    Context: {context}

    Question: {question}
    """

In [7]:
prompt = PromptTemplate.from_template(template)

In [8]:
chain = prompt | chain

In [9]:
print(chain.invoke(
    {
        "context": "The name I was given was Jitendra",
        "question": "What's my name?"
    }
))


Hello Jitendra! Your name is Jitendra.


# VectorDB

## DocArray

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

In [ ]:
vectorstores = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

retriever = vectorstores.as_retriever()

## FAISS

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(loader.load())

In [ ]:
faiss_db = FAISS.from_documents(texts, embedding=embeddings)

retriever = faiss_db.as_retriever()

## ChromDB

In [10]:
from langchain_chroma import Chroma

In [11]:
vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db", 
)

In [12]:
from uuid import uuid4

In [19]:
uuids = [str(uuid4()) for _ in range(len(documents))]

In [20]:
vector_store.add_documents(documents=documents,ids=uuids)

['c55b30c0-4685-43c3-a07f-eafb39fe6a00',
 'c18a2871-bb12-49e7-bdda-ea6385230866',
 '44bde7fc-4733-4c71-8cd2-a30987376179',
 'd9b01382-fbec-454d-8179-be45ad135969',
 'e6a9bb43-61e2-4d9f-9550-9d91496a6b31',
 '7f270b43-29f0-4ad4-bd0c-51d9e8d7cde6',
 '4cb43a66-83b5-4d2d-89d6-747353bb2bf6',
 'cddb1a3e-4216-4473-99ff-cea50afd2d0b',
 '25930199-bb56-45a3-b5e5-b914986a09d4',
 'ad6fb8a4-9e4d-4d5b-a990-ecd589db32c9',
 '6985deb2-72d7-42a7-abf3-4569505930c3',
 '5e6770cb-ccb3-4e82-9767-660ab512ddb6',
 '6386fc45-635e-429a-a9d4-16a4bf313869',
 'ee2deffb-9c0e-47d3-a397-60510e633924',
 'be77cf07-8661-4625-a6f7-2e2e75e240cd',
 '9daa8623-70ea-488c-b80d-554460933b24',
 '8f84e4ed-64dc-496c-9fb4-a58fb96140cb',
 '4ed30386-9bb8-4339-a17b-7bd51c6d4365',
 '8274bd8d-81fb-422b-9f95-f4091c4e3c42',
 '201c4bf7-520b-4161-9833-bc0d07714842',
 '93e10761-2fdf-463d-b6c2-344881184ddb',
 'd781537e-a90e-4658-a57d-00f1258719be',
 '600211fc-1487-47cf-872f-a6123e07da7f',
 '6ae8bf93-6634-4eae-87c0-d32c5253d34d',
 'be211158-5d93-

In [21]:
retriever = vector_store.as_retriever()

## Combine Model chain with Retriever VectorDB

In [22]:
chain_retriever = ({
    "context": itemgetter("question") | retriever,
    "question": itemgetter("question")
} | chain)

resp = chain_retriever.invoke({"question": "How many types of networking possible in the docker?"})

display(Markdown(resp))

The answer to your question can be found in the document under the section "Networking in Docker":

"Docker provides two networking models: host mode and container mode."

So, there are two types of networking possible in Docker:

1. Host mode: In this mode, the container's networking is exposed to the host machine's network stack. This means that the container can communicate with the host and other containers on the same network.
2. Container mode: In this mode, the container has its own networking stack, isolated from the host machine's network stack. This means that the container can only communicate with other containers in the same network, not with the host machine or other containers in different networks.

I hope this helps! Let me know if you have any further questions.